## HBI, PPGA 추출

In [17]:
import os, sys
import pickle
import pandas as pd
import numpy as np

#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr_mod, vitaldb, pleth_spi, arr

# creating histogram
print("created a new group histogram\n")
group_hist_ppga = pleth_spi.Histogram(0, 20, 1000)
group_hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

non_list = []
vital_path = 'PACU_vital/PACU1_10-12'
vital_list = os.listdir(vital_path)

srate, cnt = 100, 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG2/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('({}/{}) '.format(cnt, len(vital_list)), end=' ')
        print('already existing...loading sample... '+f_vital, end='')
        hbi_list, ppga_list = pickle.load(open(cachepath, 'rb'))
        print('done')
        
        for hbi in hbi_list:
            group_hist_hbi.learn(hbi)
        for ppga in ppga_list:
            group_hist_ppga.learn(ppga)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        if not '.vital' in f_vital:
            print('not a vital file')
            non_list.append(f_vital)
            continue
        
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        print('size of', f_vital, ':', len(vals), end='')
        
        if len(vals) == 1:
            print('no data')
            non_list.append(f_vital)
            continue
        
        print('\ndetecting peak...', end='')
        vals = arr.exclude_undefined(vals)
        
        # vals 값이 일정한 경우
        if len(np.unique(vals)) == 1:
            print('no valid data')
            non_list.append(f_vital)
            continue
        
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                group_hist_hbi.learn(hbi)
                hbi_list.append(hbi)
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                group_hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))
        

created a new group histogram

(1/678)  already existing...loading sample... PACU1_12_200115_153355.vitaldone
(2/678)  already existing...loading sample... PACU1_10_190827_113303.vitaldone
(3/678)  already existing...loading sample... PACU1_10_190429_163301.vitaldone
(4/678)  already existing...loading sample... PACU1_11_200117_155606.vitaldone
(5/678)  already existing...loading sample... PACU1_11_200114_103104.vitaldone
(6/678)  already existing...loading sample... PACU1_11_190708_161809.vitaldone
(7/678)  already existing...loading sample... PACU1_11_200715_165207.vitaldone
(8/678)  already existing...loading sample... PACU1_11_190429_120455.vitaldone
(9/678)  already existing...loading sample... PACU1_11_190514_130430.vitaldone
(10/678)  already existing...loading sample... PACU1_12_200529_153418.vitaldone
(11/678)  already existing...loading sample... PACU1_10_200303_100225.vitaldone
(12/678)  already existing...loading sample... PACU1_10_201027_103908.vitaldone
(13/678)  already 

(106/678)  already existing...loading sample... PACU1_12_190517_111404.vitaldone
(107/678)  already existing...loading sample... PACU1_10_191219_104153.vitaldone
(108/678)  already existing...loading sample... PACU1_11_200228_154436.vitaldone
(109/678)  already existing...loading sample... PACU1_10_200526_102844.vitaldone
(110/678)  already existing...loading sample... PACU1_10_200508_123320.vitaldone
(111/678)  already existing...loading sample... PACU1_10_190719_133911.vitaldone
(112/678)  already existing...loading sample... PACU1_11_200612_160718.vitaldone
(113/678)  already existing...loading sample... PACU1_12_200819_132943.vitaldone
(114/678)  already existing...loading sample... PACU1_10_200810_144911.vitaldone
(115/678)  already existing...loading sample... PACU1_11_200805_165907.vitaldone
(116/678)  already existing...loading sample... PACU1_10_190718_143135.vitaldone
(117/678)  already existing...loading sample... PACU1_10_200803_165836.vitaldone
(118/678)  already existing.

(214/678)  already existing...loading sample... PACU1_10_200316_145935.vitaldone
(215/678)  already existing...loading sample... PACU1_11_190905_145000.vitaldone
(216/678)  already existing...loading sample... PACU1_11_190828_094209.vitaldone
(217/678)  already existing...loading sample... PACU1_10_190430_120158.vitaldone
(218/678)  already existing...loading sample... PACU1_10_200529_120757.vitaldone
(219/678)  already existing...loading sample... PACU1_11_201020_133418.vitaldone
(220/678)  already existing...loading sample... PACU1_10_190502_125645.vitaldone
(221/678)  already existing...loading sample... PACU1_12_200918_133600.vitaldone
(222/678)  already existing...loading sample... PACU1_12_201030_152744.vitaldone
(223/678)  already existing...loading sample... PACU1_10_200805_171050.vitaldone
(224/678)  already existing...loading sample... PACU1_12_200306_163625.vitaldone
(225/678)  already existing...loading sample... PACU1_10_200228_130016.vitaldone
(226/678)  already existing.

detecting peak...completed...
(412/678) size of PACU1_10_200214_134908.vital : 1no data
(413/678) size of PACU1_11_201013_134512.vital : 233549
detecting peak...completed...
(414/678) size of PACU1_11_190829_135831.vital : 408376
detecting peak...completed...
(415/678) size of PACU1_12_190716_140916.vital : 285561
detecting peak...completed...
(416/678) size of PACU1_11_200302_162017.vital : 244542
detecting peak...completed...
(417/678) size of PACU1_11_200527_134117.vital : 277454
detecting peak...completed...
(418/678) size of PACU1_10_190910_153828.vital : 274550
detecting peak...completed...
(419/678) size of PACU1_11_200525_165009.vital : 222753
detecting peak...completed...
(420/678) size of PACU1_10_200805_120121.vital : 289312
detecting peak...completed...
(421/678) size of PACU1_10_190524_154757.vital : 275895
detecting peak...completed...
(422/678) size of PACU1_12_190709_123037.vital : 273789
detecting peak...completed...
(423/678) size of PACU1_11_201007_102740.vital : 225

(510/678) size of PACU1_12_190820_142443.vital : 280842
detecting peak...completed...
(511/678) size of PACU1_11_190514_093500.vital : 279743
detecting peak...completed...
(512/678) size of PACU1_12_190829_124001.vital : 285198
detecting peak...completed...
(513/678) size of PACU1_11_200506_162047.vital : 271404
detecting peak...completed...
(514/678) size of PACU1_11_200219_135646.vital : 1no data
(515/678) size of PACU1_12_200122_163312.vital : 1no data
(516/678) size of PACU1_12_200706_133148.vital : 261240
detecting peak...completed...
(517/678) size of PACU1_10_200522_104203.vital : 252891
detecting peak...completed...
(518/678) size of PACU1_12_190903_152223.vital : 266899
detecting peak...completed...
(519/678) size of PACU1_10_190820_153302.vital : 306131
detecting peak...completed...
(520/678) size of PACU1_11_190827_131853.vital : 282106
detecting peak...completed...
(521/678) size of PACU1_10_200327_131020.vital : 218816
detecting peak...completed...
(522/678) size of PACU1_

detecting peak...completed...
(608/678) size of PACU1_11_190515_150348.vital : 284116
detecting peak...completed...
(609/678) size of PACU1_10_200116_095025.vital : 288949
detecting peak...completed...
(610/678) size of PACU1_12_200603_182203.vital : 240912
detecting peak...completed...
(611/678) size of PACU1_11_190704_122836.vital : 555955
detecting peak...completed...
(612/678) size of PACU1_10_200603_164857.vital : 232563
detecting peak...completed...
(613/678) size of PACU1_10_200219_113830.vital : 1no data
(614/678) size of PACU1_10_200603_131444.vital : 221163
detecting peak...completed...
(615/678) size of PACU1_10_200512_174020.vital : 318217
detecting peak...completed...
(616/678) size of PACU1_12_190819_175604.vital : 444313
detecting peak...completed...
(617/678) size of PACU1_10_200612_133526.vital : 252276
detecting peak...completed...
(618/678) size of PACU1_11_200702_145157.vital : 273703
detecting peak...completed...
(619/678) size of PACU1_12_191022_145850.vital : 294

In [31]:
import os, sys
import pickle
import pandas as pd
import numpy as np

#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr_mod, vitaldb, pleth_spi

# creating histogram
print("created a new group histogram\n")
group_hist_ppga = pleth_spi.Histogram(0, 20, 1000)
group_hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

non_list = []
vital_path = 'PACU_vital/PACU1_10-12'
vital_list = os.listdir(vital_path)

created a new group histogram



In [47]:
%load_ext autoreload
%autoreload 2

from pyvital2 import vitaldb

f_vital = 'PACU1_10_200131_125741.vital'
#f_vital = 'PACU1_10_200710_092426.vital'
vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
#vals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
case2
case2
case2
case2
dname: SNUADCW trks.values: dict_values([])
trk: None


## PPG 파일 옮기기

In [60]:
import shutil, os

cache_path = 'cache/PPG'
file_list = os.listdir(cache_path)

'''
# 파일 위치 바꾸기
for file_name in file_list:
    if 'PACU1_12' in file_name:
        print(file_name)
        shutil.move('cache/PPG2/'+file_name, 'cache/PPG/'+file_name)
    
# 파일 이름 바꾸기
for file_name in file_list:
    if 'hbi_ppga2_' in file_name:
        mod_name = 'hbi_ppga_' +file_name[10:]
        print(file_name, '...changed')
        os.rename(os.path.join(cache_path,file_name), os.path.join(cache_path,mod_name))
'''

cnt = 0
for file_name in file_list:
    if 'hbi_ppga_' in file_name:
        cnt+=1
print(cnt)

4735


In [15]:
#vals = vitaldb.vital_recs(vital_path+'/PACU1_1_200224_200759.vital', ['SNUADCW/PLETH'], interval = 0.01).flatten()
group_hist_ppga.bins

[10575,
 11,
 8,
 63,
 25,
 30,
 102137,
 5016,
 10,
 9,
 4,
 0,
 3,
 0,
 5,
 4013,
 23,
 18,
 14,
 16521,
 35,
 49,
 129824,
 1,
 10,
 6,
 1,
 12,
 2551,
 15,
 596,
 2,
 93,
 9,
 1,
 9,
 15,
 102,
 546,
 20210,
 11317,
 19,
 37,
 178,
 0,
 490,
 19801,
 15,
 56,
 7,
 109,
 1,
 16,
 414,
 2,
 1,
 63636,
 14,
 118689,
 15,
 9,
 417,
 8,
 47020,
 21,
 13,
 12,
 7,
 5253,
 31,
 120,
 32,
 185314,
 9,
 26,
 6,
 396,
 69,
 1780,
 3,
 34,
 7,
 15,
 358,
 2946,
 10,
 3,
 3,
 30775,
 4,
 44432,
 344,
 2,
 0,
 7,
 69459,
 51,
 1610,
 32,
 338,
 5,
 10,
 7,
 3,
 12192,
 3681,
 388,
 15,
 18,
 0,
 3,
 35,
 31,
 16,
 336,
 14,
 0,
 1389,
 13,
 13,
 19,
 81871,
 361,
 8534,
 2,
 17,
 0,
 17,
 2378,
 37497,
 36325,
 14,
 32,
 88275,
 31,
 18,
 1397,
 445,
 16,
 3,
 31,
 34,
 3,
 31,
 361,
 1086,
 95820,
 15631,
 50,
 35,
 24,
 26,
 339,
 48,
 9391,
 0,
 1387,
 13389,
 8,
 35254,
 355,
 47,
 9033,
 1,
 37424,
 19,
 3,
 373,
 34,
 72215,
 5,
 10525,
 4,
 9,
 10,
 1957,
 11,
 5,
 7,
 2,
 3,
 0,
 2,
 31

In [7]:
vital_path = 'PACU_vital/PACU1_10-12'
vital_list = os.listdir(vital_path)
len(vital_list)

678

In [8]:
2048+1453+1096+678

5275

In [65]:
import os, sys
import pickle
import pandas as pd
import numpy as np

#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr_mod, vitaldb, pleth_spi, arr

# creating histogram
print("created a new group histogram\n")
group_hist_ppga = pleth_spi.Histogram(0, 100, 1000)
group_hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

non_list = []
vital_path = 'PACU_vital/PACU1_7-9'
vital_list = os.listdir(vital_path)

srate, cnt = 100, 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG_mod/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('({}/{}) '.format(cnt, len(vital_list)), end=' ')
        print('already existing...loading sample... '+f_vital, end='')
        hbi_list, ppga_list = pickle.load(open(cachepath, 'rb'))
        print('done')
        
        #for hbi in hbi_list:
            #group_hist_hbi.learn(hbi)
        #for ppga in ppga_list:
            #group_hist_ppga.learn(ppga)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        if not '.vital' in f_vital:
            print('not a vital file')
            non_list.append(f_vital)
            continue
        
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        print('size of', f_vital, ':', len(vals), end='')
        
        if len(vals) == 1:
            print('no data')
            non_list.append(f_vital)
            continue
        
        print('\ndetecting peak...', end='')
        vals = arr_mod.exclude_undefined(vals)
        
        # vals 값이 일정한 경우
        if len(np.unique(vals)) == 1:
            print('no valid data')
            non_list.append(f_vital)
            continue
        
        minlist, maxlist = arr_mod.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                #group_hist_hbi.learn(hbi)
                hbi_list.append(hbi)
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                #group_hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))
        

created a new group histogram

(1/1096) size of PACU1_9_200217_100251.vital : 1no data
(2/1096) size of PACU1_7_200205_104734.vital : 1no data
(3/1096) size of PACU1_8_200715_174003.vital : 266685
detecting peak...completed...
(4/1096) size of PACU1_9_201029_091818.vital : 235484
detecting peak...completed...
(5/1096) size of PACU1_7_200722_091933.vital : 233531
detecting peak...completed...
(6/1096) size of PACU1_7_190514_141137.vital : 277267
detecting peak...completed...
(7/1096) size of PACU1_7_200514_121133.vital : 255674
detecting peak...completed...
(8/1096) size of PACU1_8_200602_173332.vital : 192028
detecting peak...completed...
(9/1096) size of PACU1_8_190820_171847.vital : 276728
detecting peak...completed...
(10/1096) size of PACU1_7_200717_162235.vital : 255432
detecting peak...completed...
(11/1096) size of PACU1_8_200220_171139.vital : 1no data
(12/1096) size of PACU1_9_201013_144641.vital : 241264
detecting peak...completed...
(13/1096) size of PACU1_7_200311_091556.vi

(101/1096) size of PACU1_8_200923_090419.vital : 300202
detecting peak...completed...
(102/1096) size of PACU1_8_200115_173634.vital : 256158
detecting peak...completed...
(103/1096) size of PACU1_9_200914_163331.vital : 268379
detecting peak...completed...
(104/1096) size of PACU1_8_200710_162631.vital : 257247
detecting peak...completed...
(105/1096) size of PACU1_9_200626_162342.vital : 205212
detecting peak...completed...
(106/1096) size of PACU1_7_191023_123318.vital : 256279
detecting peak...completed...
(107/1096) size of PACU1_7_200115_104704.vital : 278059
detecting peak...completed...
(108/1096) size of PACU1_9_200917_114802.vital : 224949
detecting peak...completed...
(109/1096) size of PACU1_9_200109_150043.vital : 320288
detecting peak...completed...
(110/1096) size of PACU1_7_200703_145656.vital : 246962
detecting peak...completed...
(111/1096) size of PACU1_7_200305_142705.vital : 366752
detecting peak...completed...
(112/1096) size of PACU1_9_200602_143200.vital : 28120

detecting peak...completed...
(200/1096) size of PACU1_8_200103_103213.vital : 297903
detecting peak...completed...
(201/1096) size of PACU1_7_200115_210908.vital : 245147
detecting peak...completed...
(202/1096) size of PACU1_7_200526_155953.vital : 290038
detecting peak...completed...
(203/1096) size of PACU1_7_190718_132645.vital : 406319
detecting peak...completed...
(204/1096) size of PACU1_8_200220_131327.vital : 1no data
(205/1096) size of PACU1_9_200303_120756.vital : 252407
detecting peak...completed...
(206/1096) size of PACU1_7_200302_110706.vital : 269105
detecting peak...completed...
(207/1096) size of PACU1_7_200318_162827.vital : 242485
detecting peak...completed...
(208/1096) size of PACU1_9_200601_162200.vital : 244905
detecting peak...completed...
(209/1096) size of PACU1_7_200117_171905.vital : 246357
detecting peak...completed...
(210/1096) size of PACU1_7_200204_181236.vital : 1no data
(211/1096) size of PACU1_9_190709_093004.vital : 311187
detecting peak...complet

detecting peak...completed...
(299/1096) size of PACU1_7_200616_121614.vital : 228570
detecting peak...completed...
(300/1096) size of PACU1_8_190712_103040.vital : 268016
detecting peak...completed...
(301/1096) size of PACU1_7_200903_134519.vital : 251723
detecting peak...completed...
(302/1096) size of PACU1_9_201027_130800.vital : 271041
detecting peak...completed...
(303/1096) size of PACU1_7_201019_125200.vital : 224802
detecting peak...completed...
(304/1096) size of PACU1_8_190829_091653.vital : 271888
detecting peak...completed...
(305/1096) size of PACU1_9_200129_092229.vital : 1no data
(306/1096) size of PACU1_7_200203_094632.vital : 1no data
(307/1096) size of PACU1_8_200513_145119.vital : 262208
detecting peak...completed...
(308/1096) size of PACU1_9_200925_091850.vital : 242727
detecting peak...completed...
(309/1096) size of PACU1_8_200331_095710.vital : 245147
detecting peak...completed...
(310/1096) size of PACU1_7_201008_095814.vital : 235620
detecting peak...complet

detecting peak...completed...
(399/1096) size of PACU1_9_200325_171903.vital : 234378
detecting peak...completed...
(400/1096) size of PACU1_7_200727_143834.vital : 256642
detecting peak...completed...
(401/1096) size of PACU1_9_200526_103443.vital : 253738
detecting peak...completed...
(402/1096) size of PACU1_9_190703_173328.vital : 319441
detecting peak...completed...
(403/1096) size of PACU1_8_200225_172129.vital : 1no data
(404/1096) size of PACU1_8_190717_135700.vital : 265838
detecting peak...completed...
(405/1096) size of PACU1_9_200325_105720.vital : 253859
detecting peak...completed...
(406/1096) size of PACU1_7_200515_153753.vital : 273141
detecting peak...completed...
(407/1096) size of PACU1_9_190528_105044.vital : 60310
detecting peak...completed...
(408/1096) size of PACU1_8_200701_105336.vital : 262102
detecting peak...completed...
(409/1096) size of PACU1_7_200813_165138.vital : 275760
detecting peak...completed...
(410/1096) size of PACU1_8_190514_143339.vital : 2695

...

detecting peak...completed...
(696/1096) size of PACU1_8_190712_153517.vital : 114830
detecting peak...completed...
(697/1096) size of PACU1_9_200320_100213.vital : 244292
detecting peak...completed...
(698/1096) size of PACU1_9_200602_132222.vital : 259062
detecting peak...completed...
(699/1096) size of PACU1_7_200722_103743.vital : 238855
detecting peak...completed...
(700/1096) size of PACU1_7_200224_140725.vital : 1no data
(701/1096) size of PACU1_8_190904_135952.vital : 325854
detecting peak...completed...
(702/1096) size of PACU1_7_200825_104427.vital : 268984
detecting peak...completed...
(703/1096) size of PACU1_9_190529_111324.vital : 255563
detecting peak...completed...
(704/1096) size of PACU1_9_201026_135006.vital : 229065
detecting peak...completed...
(705/1096) size of PACU1_7_200721_114447.vital : 259395
detecting peak...completed...
(706/1096) size of PACU1_8_200122_091942.vital : 1no data
(707/1096) size of PACU1_7_190708_144853.vital : 280963
detecting peak...complet

detecting peak...completed...
(795/1096) size of PACU1_9_200924_115216.vital : 366147
detecting peak...completed...
(796/1096) size of PACU1_9_200910_144614.vital : 295846
detecting peak...completed...
(797/1096) size of PACU1_8_200107_171716.vital : 248172
detecting peak...completed...
(798/1096) size of PACU1_8_200806_102639.vital : 239924
detecting peak...completed...
(799/1096) size of PACU1_7_190909_132756.vital : 284593
detecting peak...completed...
(800/1096) size of PACU1_9_200612_094447.vital : 244800
detecting peak...completed...
(801/1096) size of PACU1_9_200911_160825.vital : 271404
detecting peak...completed...
(802/1096) size of PACU1_8_200217_132002.vital : 1no data
(803/1096) size of PACU1_7_200309_111352.vital : 294999
detecting peak...completed...
(804/1096) size of PACU1_7_200710_123802.vital : 237869
detecting peak...completed...
(805/1096) size of PACU1_9_200123_145052.vital : 1no data
(806/1096) size of PACU1_7_200806_154537.vital : 241275
detecting peak...complet

detecting peak...completed...
(895/1096) size of PACU1_9_201016_123439.vital : 233894
detecting peak...completed...
(896/1096) size of PACU1_8_200317_125607.vital : 253133
detecting peak...completed...
(897/1096) size of PACU1_9_201007_174038.vital : 226364
detecting peak...completed...
(898/1096) size of PACU1_9_200715_180302.vital : 128745
detecting peak...completed...
(899/1096) size of PACU1_7_200528_172604.vital : 226099
detecting peak...completed...
(900/1096) size of PACU1_8_200219_164737.vital : 1no data
(901/1096) size of PACU1_8_200728_111611.vital : 287497
detecting peak...completed...
(902/1096) size of PACU1_7_200122_134621.vital : 1no data
(903/1096) size of PACU1_7_200623_102728.vital : 308473
detecting peak...completed...
(904/1096) size of PACU1_7_200710_114423.vital : 271888
detecting peak...completed...
(905/1096) size of PACU1_8_200120_124416.vital : 237040
detecting peak...completed...
(906/1096) size of PACU1_8_200115_122326.vital : 133109
detecting peak...complet

detecting peak...completed...
(994/1096) size of PACU1_9_200520_102624.vital : 108731
detecting peak...completed...
(995/1096) size of PACU1_7_200810_093356.vital : 245994
detecting peak...completed...
(996/1096) size of PACU1_9_200320_164448.vital : 246478
detecting peak...completed...
(997/1096) size of PACU1_9_200114_154900.vital : 270241
detecting peak...completed...
(998/1096) size of PACU1_7_200213_191646.vital : 1no data
(999/1096) size of PACU1_8_190709_132151.vital : 237645
detecting peak...completed...
(1000/1096) size of PACU1_7_200522_161859.vital : 239259
detecting peak...completed...
(1001/1096) size of PACU1_9_190502_154658.vital : 349988
detecting peak...completed...
(1002/1096) size of PACU1_7_190918_162806.vital : 127312
detecting peak...completed...
(1003/1096) size of PACU1_7_190821_104558.vital : 292326
detecting peak...completed...
(1004/1096) size of PACU1_8_200320_154240.vital : 236478
detecting peak...completed...
(1005/1096) size of PACU1_8_200921_130534.vital

detecting peak...completed...
(1092/1096) size of PACU1_7_200522_151914.vital : 232926
detecting peak...completed...
(1093/1096) size of PACU1_8_200211_164059.vital : 1no data
(1094/1096) size of PACU1_7_190430_174537.vital : 271883
detecting peak...completed...
(1095/1096) size of PACU1_9_200708_130730.vital : 263418
detecting peak...completed...
(1096/1096) size of PACU1_7_200519_153300.vital : 234825
detecting peak...completed...
